In [4]:
import numpy as np
from datetime import datetime

np.random.seed(0)

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = float(tmp_val) - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [11]:
class LogicGate:
    def __init__(self, x_data, t_data, i_node, h_node, o_node, lr, iter_count):
        if x_data.ndim == 1:    # vector
            self.x_data = x_data.reshape((len(x_data)//i_node), i_node)
            self.t_data = t_data.reshape((len(t_data)//o_node), o_node)
        elif x_data.ndim == 2:  # matrix
            self.x_data = x_data
            self.t_data = t_data

        self.learning_rate = lr
        self.iteration_count = iter_count
        
        self.W2 = np.random.rand(i_node, h_node)
        self.b2 = np.random.rand(1)
        
        self.W3 = np.random.rand(h_node, o_node)
        self.b3 = np.random.rand(1)
        
        print("LogicGate Object is created.\n")
        
    def __feed_forward(self):
        delta = 1e-7
        
        z2 = np.dot(self.x_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        return -np.sum((self.t_data*np.log10(a3 + delta)) + ((1 - self.t_data)*np.log10((1 - a3) + delta)))

    def __loss_val(self):
        delta = 1e-7
        
        z2 = np.dot(self.x_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        return -np.sum((self.t_data*np.log10(a3 + delta)) + ((1 - self.t_data)*np.log10((1 - a3) + delta)))
    
    def train(self):
        f = lambda x : self.__feed_forward()
        
        print("Initial loss value = ", self.__loss_val(), '\n')
        
        start_time = datetime.now()

        for step in range(self.iteration_count):
            self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
            self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
            self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
            self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
            
            if step % (int)(0.05*self.iteration_count) == 0:
                print("step = ", step, " : ", "loss value = ", self.__loss_val())
        
        end_time = datetime.now()

        print("")
        print("Elapsed Time => ", end_time - start_time)
        
    def predict(self, test_data):
        z2 = np.dot(test_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
        
        if a3 >= 0.5:
            result = 1
        else:
            result = 0
    
        return y, result
    
    def accuracy(self, test_data):
        input_data = test_data[ :, 0:-1]
        target_data = test_data[ :, -1:]
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_val = len(matched_list) / len(input_data)
        
        return accuracy_val

In [12]:
xdata = np.array([1, 1, 0, 1, 1, 0, 0, 0])
tdata = np.array([1, 0, 0, 0])

AND_obj = LogicGate(xdata, tdata, 2, 3, 1, 1e-1, 101)

AND_obj.train()

AND_obj.accuracy(np.array([[1, 1, 1], [0, 1, 0], [1, 0, 0], [0, 0, 0]]))

LogicGate Object is created.

Initial loss value =  1.7877206619070942 

step =  0  :  loss value =  1.6592097253357312
step =  5  :  loss value =  1.2709148721500998
step =  10  :  loss value =  1.1150083716180754
step =  15  :  loss value =  1.0505689766120812
step =  20  :  loss value =  1.0223596882886479
step =  25  :  loss value =  1.0091295936755613
step =  30  :  loss value =  1.0023648899841537
step =  35  :  loss value =  0.9984917401873284
step =  40  :  loss value =  0.9959501068642351
step =  45  :  loss value =  0.9940381155796207
step =  50  :  loss value =  0.9924325122103983
step =  55  :  loss value =  0.9909814420796069
step =  60  :  loss value =  0.989612082933921
step =  65  :  loss value =  0.9882885192257447
step =  70  :  loss value =  0.9869922110620084
step =  75  :  loss value =  0.9857128340478569
step =  80  :  loss value =  0.9844439496960229
step =  85  :  loss value =  0.9831809489653032
step =  90  :  loss value =  0.9819200727641988
step =  95  :  los

0.75